## Hyper parameter tuning by SageMaker

1. Prepare Requirements
2. 

**Reference**

* [Hyperparameter Tuning using SageMaker PyTorch Container](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/pytorch_mnist/hpo_pytorch_mnist.ipynb)

## Os level setting

change default gcc `gcc -> /etc/alternatives/gcc` to `/usr/bin/gcc48`

```
sudo rm /usr/bin/gcc
sudo ln -s /usr/bin/gcc48 /usr/bin/gcc
```

Install `clang`

```
sudo yum install clang
```


## Prepare Requirements

In [15]:
!cd ../
!pip install pipenv

You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
!pipenv install --python 3.6

Virtualenv already exists!
Removing existing virtualenv…
Creating a virtualenv for this project…
Pipfile: /home/ec2-user/SageMaker/allennlp-sagemaker-tuning/Pipfile
Using /home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/python (3.6.8) to create virtualenv…
⠇ Creating virtual environment...Using base prefix '/home/ec2-user/anaconda3/envs/JupyterSystemEnv'
New python executable in /home/ec2-user/.local/share/virtualenvs/allennlp-sagemaker-tuning-Bpkjlbyp/bin/python
Installing setuptools, pip, wheel...
done.
Running virtualenv with interpreter /home/ec2-user/anaconda3/envs/JupyterSystemEnv/bin/python

✔ Successfully created virtual environment! 
Virtualenv location: /home/ec2-user/.local/share/virtualenvs/allennlp-sagemaker-tuning-Bpkjlbyp
  $ pipenv --rm and rebuilding the virtual environment may resolve the issue.
  $ pipenv check will surely fail.
Installing dependencies from Pipfile.lock (44ebd7)…
An error occurred while installing jsonnet==0.12.1 ; sys_platform != 'win32' --hash=sh